# Objectives

1. Build a deep learning model to classify infected cells
2. Build a simple application using streamlit to classify a given image and highlight the hotspots in the image that influence the prediction
    - The hotspots have to be overlayed on the image and displayed in the streamlit app
    - Include a video demo of the app using a screen recorder
    - Bonus: deploy the streamlit app on a cloud platform (eg: heroku)
3. Detail your approach and findings from the dataset in a 2-4 page technical report using a prefered format. Provide the necessary details to rationalize your assumptions and choice of methods.
4. Submit an archive containing the code and the report.
    - If you used a kaggle kernel, publish it and share the kernel link

# Evaluation criteria

The submissions will be evaluated on the following criteria

- The different approaches explored and the overall efficiency of the solution
- The working of the application for classification and explanation
- Effective communication of results through the technical paper
- Performance of the final model

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from keras.preprocessing.image import ImageDataGenerator
#print(os.listdir("cell_images/cell_images"))

In [2]:
img_width= 128
img_height= 128

In [3]:
datagen= ImageDataGenerator(rescale=1/255.0,validation_split=0.2)

In [4]:
train_data_generator= datagen.flow_from_directory(directory='cell_images/cell_images',target_size=(img_width,img_height),
                                                  class_mode='binary',batch_size=16, subset='training')

Found 22048 images belonging to 2 classes.


In [5]:
validation_data_generator= datagen.flow_from_directory(directory='cell_images/cell_images',target_size=(img_width,img_height),
                                                  class_mode='binary',batch_size=16, subset='validation')

Found 5510 images belonging to 2 classes.


In [6]:
train_data_generator.labels

array([0, 0, 0, ..., 1, 1, 1])

# CNN

In [7]:
from keras import Sequential
from keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPool2D
from tensorflow.keras.optimizers import Adam

In [8]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 30, 30, 32)        0

In [10]:
model.compile(optimizer=Adam(),
             metrics=['Accuracy'],
             loss='binary_crossentropy')

In [11]:
history = model.fit(train_data_generator,steps_per_epoch=len(train_data_generator),
         epochs=5,validation_data=validation_data_generator,
         validation_steps=len(validation_data_generator))

Epoch 1/5
1378/1378 [==============================] - 186s 134ms/step - loss: 0.4241 - Accuracy: 0.8082 - val_loss: 0.2001 - val_Accuracy: 0.9285
Epoch 2/5
1378/1378 [==============================] - 183s 133ms/step - loss: 0.1866 - Accuracy: 0.9432 - val_loss: 0.1696 - val_Accuracy: 0.9454
Epoch 3/5
1378/1378 [==============================] - 179s 130ms/step - loss: 0.1622 - Accuracy: 0.9535 - val_loss: 0.2017 - val_Accuracy: 0.9419
Epoch 4/5
1378/1378 [==============================] - 189s 137ms/step - loss: 0.1515 - Accuracy: 0.9560 - val_loss: 0.1906 - val_Accuracy: 0.9423
Epoch 5/5
1378/1378 [==============================] - 186s 135ms/step - loss: 0.1408 - Accuracy: 0.9563 - val_loss: 0.1919 - val_Accuracy: 0.9426
